In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip uninstall --y typing
!pip install nvtx dask_cuda
!pip install git+https://github.com/NVIDIA/NVTabular.git@4c92dffac4354d816178264bcfcdec722db2ec1c

In [ ]:
import os
from time import time
import re
import glob
import warnings
# import cudf
import gc

# tools for data preproc/loading
import torch
# import rmm
# import nvtabular as nvt
# from nvtabular.ops import Normalize,  Categorify,  LogOp, FillMissing, Clip, get_embedding_sizes
# from nvtabular.loader.torch import TorchAsyncItr, DLDataLoader
# from nvtabular.utils import device_mem_size, get_rmm_size
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# tools for training
from fastai.basics import Learner

from fastai.tabular.all import *
from fastai.tabular.model import TabularModel
from fastai.tabular.data import TabularDataLoaders
from fastai.metrics import accuracy
from fastai.callback.progress import ProgressCallback
from collections import defaultdict
from numba import njit
from scipy import stats
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from kmodes import kmodes
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

# import dask
# import dask.dataframe as dd
import pandas as pd
import numpy as np
gc.collect()

  ################################################################################################################

# class CFG:
#     TRAIN_SIZE = 0.2
#     MIN_WEIGHT = 0
#     NROWS = None
#     FILL_NA = -999
    
# # -75% of time space took by train.csv so we can train faster models
# # and avoid RAM errors

# features_columns = ["feature_%d" % i for i in range(130)]
# columns_dtypes = {}
# for column in features_columns:
#     columns_dtypes[column] = "float16"
# columns_dtypes["resp_1"] = "float16"
# columns_dtypes["resp_2"] = "float16"
# columns_dtypes["resp_3"] = "float16"
# columns_dtypes["resp_4"] = "float16"
# columns_dtypes["resp"] = "float16"

# print("Loading dataset...")
# dataset = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv", delimiter=",", nrows=CFG.NROWS, dtype=columns_dtypes)
# dataset = dataset[dataset.weight > CFG.MIN_WEIGHT]
# print("Done!")

# print("Splitting train/test dataset...")
# train_number_items = int(dataset.shape[0] * CFG.TRAIN_SIZE)
# train = dataset[:train_number_items]
# test = dataset[train_number_items + 1:]
# print("Done!")

# print("Filling NaN values...")
# train = train.fillna(CFG.FILL_NA)
# test = test.fillna(CFG.FILL_NA)
# print("Done.")

# print("Preparing X and y...")
# X_train = train[features_columns].to_numpy()
# X_test = test[features_columns].to_numpy()
# y_train = np.where(train["resp"] > 0, 1, 0)
# y_test = np.where(test["resp"] > 0, 1, 0)
# d_train = train["date"].to_numpy()
# d_test = test["date"].to_numpy()
# w_train = train["weight"].to_numpy()
# w_test = test["weight"].to_numpy()
# r_train = train["resp"].to_numpy()
# r_test = test["resp"].to_numpy()
# resp_train = train[["resp_1", "resp_2", "resp_3", "resp_4"]].to_numpy()
# resp_test = test[["resp_1", "resp_2", "resp_3", "resp_4"]].to_numpy()

# resp_label_train = np.where(resp_train > 0, 1, 0)
# resp_label_test = np.where(resp_test > 0, 1, 0)

# print("Done!")

# print("Deleting unused variables...")
# del dataset
# del train
# del test
# print("Done!")

# print("Train/test sizes: %d/%d" % (len(X_train), len(X_test)))

In [ ]:
js=pd.read_csv("../input/jane-street-market-prediction/train.csv")

In [ ]:
js.columns

In [ ]:
drop_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4','ts_id']

In [ ]:
js=js[js.weight>0]
js=js[js.date>85]
js['action'] = (  (js['resp_1'] > 0 ) & (js['resp_2'] > 0 ) & (js['resp_3'] > 0 ) & (js['resp_4'] > 0 ) &  (js['resp'] > 0 )   ).astype('int')
js.drop(drop_cols, inplace=True, axis=1)
js.reset_index(inplace=True)
js.to_feather('js')

In [ ]:
js.drop(['index'], inplace=True, axis=1)
js.to_feather('js')

In [ ]:
import pathlib
path = pathlib.Path("/kaggle/input/jane-street-market-prediction")

In [ ]:
js.head()

In [ ]:
dic1 = dic.set_index('Unnamed: 0').to_dict()
dic.drop(dic.index,inplace=True)
dic1=dic1['0']
train = pd.read_csv(path/"train.csv",dtype=dic1)

In [ ]:
OUTPUT_DATA_DIR='/kaggle/working/' 

In [ ]:
train_paths = ['../input/train-and-valid/train']
valid_paths = ['../input/train-and-valid/valid']

In [ ]:
train = pd.read_parquet('../input/train-and-valid/train')
valid = pd.read_parquet('../input/train-and-valid/valid')

In [ ]:
gc.collect()

In [ ]:
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
feats=[f'feature_{i}' for i in range(130)]
con_names = ['weight'] + resp_cols + feats 
cat_names = [x for x in train.columns if '_na' in x]
cat_names= ['date'] + cat_names

In [ ]:
train=train[train.weight>0]
train=train[train.date>85]
valid=valid[valid.weight>0]

In [ ]:
len(train)

In [ ]:
len(valid)

In [ ]:
data=pd.concat([train,valid])

In [ ]:
data.head()

In [ ]:
data.drop('index',axis=1,inplace=True)

In [ ]:
js.reset_index(inplace=True)

In [ ]:
js.to_feather('js') 

In [ ]:
for i in range(1,4):
    print(i)

In [ ]:
t.head()

In [ ]:
len(t)

In [ ]:
len(t)

In [ ]:
gc.collect()

In [ ]:
tnacol = train.columns[train.isnull().any()]
tna = train[tnacol].isnull()
nat={}
for c in tnacol:
    nat[c]= c+"_na"
tna.rename(mapper=nat,inplace=True)

In [ ]:
vnacol = valid.columns[valid.isnull().any()]
vna = valid[vnacol].isnull()
nav={}
for c in vnacol:
    nav[c]= c+"_na"
vna.rename(mapper=nav,inplace=True)    

In [ ]:
#Fn to reduce memory usage significantly
def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    

In [ ]:
# valid = valid.astype({c: np.float32 for c in valid.select_dtypes(include='float64').columns}) #limit memory use
# valid['action'] =  (  (valid['resp_1'] > 0 ) & (valid['resp_2'] > 0 ) & (valid['resp_3'] > 0 ) & (valid['resp_4'] > 0 ) &  (valid['resp'] > 0 )   ) & (valid['weight'] > 0).astype('int')

In [ ]:
# train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
# train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   )& (train['weight'] > 0).astype('int')


In [ ]:
def eval_metric(dataset):
    p = []
    for d in dataset.date.unique():
        p.append(np.sum(dataset[dataset.date==d].weight*dataset[dataset.date==d].resp*dataset[dataset.date==d].action))
    t = (np.sum(p)*np.sqrt(250))/(np.sqrt(np.sum(p**2))*len(dataset.date.unique()))
    u = min(max(t,0),6)*np.sum(p)
    return u


In [ ]:

def fillna_npwhere_njit(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [ ]:
t=t.to_numpy()

In [ ]:
gc.collect()

In [ ]:
l = [i for i in range(130)]
l

In [ ]:
dic

In [ ]:
di

In [ ]:
di = {}
for i in range(130):
    di[i]=f'feature_{i}'

In [ ]:

data.rename(columns = di, inplace = True)
# valid.rename(columns = di, inplace = True)

In [ ]:
data.feature_77

In [ ]:

def KNNimp(dataset):
    imputer = KNNImputer()
    dataset = imputer.fit_transform(dataset)

In [ ]:
fillm(train)

In [ ]:



#Fill Na using numpy numba : fillna_npwhere_njit
@njit(parallel=True, fastmath=True)
def cluster_missing(dataset):
    array = dataset.to_numpy()
#     dataset.interpolate(method ='quadratic', axis=0, inplace=True)
    d=dataset.isna().sum().sort_values(ascending=False).to_dict()
    g=defaultdict(list)
    for k in d.keys():
        if (d[k]!=0):
            g[d[k]].append(k)
            p=d[k]
        else:
            break;
    return g
#     for k in g.keys():
#         imp = IterativeImputer(sample_posterior=True)
#         dataset[g[k]]=imp.fit_transform(dataset[g[k]])
#         gc.collect()
#         gc.collect()
#     print(dataset.isnull().sum().sum())
# def cluster_tags(dataset):
    

def spearman(df):
    m = df.corr()
    return m;
#     sx,sy=np.where(s>0.5)
#     for  i in range(len(sx)):
#         if(sx[i]>sy[i]):

# Using kmeans to cluster the features based on their correlation
corr_feat_mtx = spearman(t).to_numpy()
kmeans = KMeans(n_clusters = 20, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
corr_feat_labels = kmeans.fit_predict(corr_feat_mtx)

# Preparing a dataframe to collect some cluster stats
# corr_feat_clust_df = pd.DataFrame(np.c_[feat_names, corr_feat_labels])
# corr_feat_clust_df.columns = ["feature", "cluster"]
# corr_feat_clust_df['feat_list'] = corr_feat_clust_df.groupby(["cluster"]).transform(lambda x: ', '.join(x))
# corr_feat_clust_df = corr_feat_clust_df.groupby(["cluster", "feat_list"]).size().reset_index(name = 'feat_count')
# corr_feat_clust_df
    





In [ ]:
gc.collect()

In [ ]:
tag1.isna().sum().sum()

In [ ]:
tag1=tag1.astype('float')

In [ ]:
x2=corr_feat_labels

In [ ]:
pip install missingno

In [ ]:
corrclus = defaultdict(list)
for i in range(130):
    f='feature_{}'.format(i)
    corrclus[corr_feat_labels[i]].append(f)

In [ ]:
corrclus

In [ ]:
# Clustering Using Tags, Missing, Distribution, Pearson Correlation 
g = defaultdict(list)
g['a'] = [1,2,9,10,15,16,42,43,45,46,48,49]
g['b'] = [3,4,5,6,7,8,11,12,14]
g['c'] = [17,18,19,20,21,22,23,24,25,26]
g['d'] = [27,28,29,30,31,32,33,34,35,36,37,39,40]
g['e'] = [41,44,51,52,53,55,56,57,58,59,60,61]
g['f'] = [42,43,45]
g['g'] = [46,48,49,50,86,87,88]
g['h'] = [62,63,65,66]
g['i'] = [64,67,68]
g['j'] = [65,66]
g['k'] = [69,70,71]
g['l'] = [72,78,84,90,93,96,99,102,105,108,114]
g['m'] = [73,74,75,76,78,79,80,81,82]
g['n'] = [120,122,124,126,128]
g['o'] = [121,123,125,127,129]
g['p'] = [89,92,98,101,104,109,113,116]
g['q'] = [95,107,119]
g['r'] = [108,110,111,112,115,117,118]
g['s'] = [92,93,94,99,100,105]
g['t'] = [85,91,97,100,103,106]
outf = [13,38,47,54,77,83]

In [ ]:
train.drop(outf,axis=1,inplace=True)
valid.drop(outf,axis=1,inplace=True)

In [ ]:
lm = [None]*130
for x in g.keys():
    for i in g[x]:
        lm[i]=x
        
        

In [ ]:
pip install --no-cache-dir --upgrade autoimpute

In [ ]:
pip install autoimpute

In [ ]:
from autoimpute.analysis import MiLinearRegression

In [ ]:
# @njit(parallel=True, fastmath=True)
def wt(dataset):
    return dataset.corr().to_numpy()
def fillm(dataset):
    for k in g.keys():
        lr = LinearRegression()
        imp = IterativeImputer(estimator=lr, verbose=2, max_iter=50, tol=1e-10, imputation_order='ascending')
        if (dataset=='valid') : imp.initial_startegy=train[g[k]].mean()
        dataset[g[k]]=imp.fit_transform(dataset[g[k]])
        gc.collect()
        gc.collect()
    print(dataset.isnull().sum().sum())

In [ ]:
gc.collect()

In [ ]:
fillm(valid)

In [ ]:
len(valid)

In [ ]:
len(train)

In [ ]:
gc.collect()

In [ ]:
train[g['a']]

In [ ]:
# GMM Clustering
def gm(t):
    from sklearn.mixture import GaussianMixture
    gmm = GaussianMixture(n_components=4)
    gmm.fit(t)

    #predictions from gmm
    labels = gmm.predict(t)
    frame = pd.DataFrame(t)
    frame['cluster'] = labels
    frame.columns = ['Weight', 'Height', 'cluster']

    color=['blue','green','cyan', 'black']
    for k in range(0,4):
        t = frame[frame["cluster"]==k]
        plt.scatter(t["Weight"],t["Height"],c=color[k])
    plt.show()


            
    



In [ ]:
train_w0 = train[train.weight==0]
train = train[train.weight>0]

In [ ]:
# fillm(train)
reduce_memory_usage(train)

In [ ]:
valid_w0 = valid[valid.weight==0]
valid = valid[valid.weight>0]

In [ ]:
gc.collect()

In [ ]:
# fillm(valid)
reduce_memory_usage(valid)
gc.collect()

In [ ]:
len(train)

In [ ]:
fillm(train)
fillm(valid)

In [ ]:
procs = [Categorify, FillMissing, Normalize]
y_names = 'salary'
y_block = CategoryBlock()

In [ ]:
to = TabularPandas(df, procs=procs, cat_names=cat_names, cont_names=con_names,
                   y_names=y_names, y_block=y_block, splits=splits)

In [ ]:
train[train.date==360]

In [ ]:
valid = train[1683632:]

In [ ]:
train=train[:1683632]

In [ ]:

len(valid)

In [ ]:
gc.collect()

In [ ]:
train.interpolate(method ='quadratic', axis=0, inplace=True)

In [ ]:
av

In [ ]:
g479=['feature_28','feature_27','feature_18','feature_17']
g476=['feature_102','feature_7','feature_78','feature_72','feature_84','feature_108','feature_96','feature_90','feature_114','feature_8'] 
g99=['feature_121','feature_120','feature_55']
g74=['feature_21','feature_32','feature_31','feature_22']
g72=['feature_92','feature_104','feature_80','feature_98','feature_116','feature_86','feature_74','feature_110','feature_11','feature_12']
g71=['feature_121','feature_120']
g48 = ['feature_55']

In [ ]:
gc.collect()

# Implementing GMM

In [ ]:
from sklearn.mixture import GaussianMixture


In [ ]:
train_y = (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')
valid_y =  (  (valid['resp_1'] > 0 ) & (valid['resp_2'] > 0 ) & (valid['resp_3'] > 0 ) & (valid['resp_4'] > 0 ) &  (valid['resp'] > 0 )   ).astype('int')

In [ ]:
gmm = GaussianMixture(n_components=2,max_iter=200).fit(train)
gc.collect()

In [ ]:
gm_preds=gmm.predict(valid)


In [ ]:
gm_preds

In [ ]:
len(gm_preds[gm_preds==valid_y])/len(gm_preds)

In [ ]:
arr = [1]*len(gm_preds)

In [ ]:
gm_preds = abs(gm_preds - arr) 

In [ ]:
gm_preds

In [ ]:
traindf = pd.concat([train,valid])

In [ ]:
len(traindf)

In [ ]:
valid.interpolate(method ='quadratic', axis=0, inplace=True)

In [ ]:
av=valid.isna().sum().sort_values(ascending=False).to_dict()

In [ ]:
av

In [ ]:
cat_features = cat_names >> Categorify()
con_features = con_names >> Normalize()

In [ ]:
features = cat_features+con_features+['action']

In [ ]:
train[con_names]=train[con_names].astype('float32')
valid[con_names]=valid[con_names].astype('float32')

In [ ]:
workflow = nvt.Workflow(features)

In [ ]:
train_dataset = nvt.Dataset(train)
valid_dataset = nvt.Dataset(valid)

In [ ]:
gc.collect()

In [ ]:


workflow.fit(train_dataset)



In [ ]:
OUTPUT_DATA_DIR='./'

In [ ]:
output_train_dir = os.path.join(OUTPUT_DATA_DIR, 'train/')
output_valid_dir = os.path.join(OUTPUT_DATA_DIR, 'valid/')
! mkdir -p $output_train_dir
! mkdir -p $output_valid_dir

In [ ]:


workflow.transform(train_dataset).to_parquet(output_path=output_train_dir,
                                             shuffle=nvt.io.Shuffle.PER_PARTITION, 
                                             out_files_per_proc=5)


In [ ]:
gc.collect()

In [ ]:


workflow.transform(valid_dataset).to_parquet(output_path=output_valid_dir, out_files_per_proc=5)



In [ ]:
len(train)+len(valid)

In [ ]:
gc.collect()

In [ ]:
BATCH_SIZE = int(os.environ.get('BATCH_SIZE', 1024*16))           # Batch-size for training neural networks
PARTS_PER_CHUNK = int(os.environ.get('PARTS_PER_CHUNK', 1))

In [ ]:
!nvidia-smi

In [ ]:
gc.collect()


In [ ]:
train_data_itrs = TorchAsyncItr(
    train_dataset,
    batch_size=BATCH_SIZE,
    cats=cat_names,
    conts=con_names,
    labels=['action'],
    parts_per_chunk=PARTS_PER_CHUNK
)
valid_data_itrs = TorchAsyncItr(
    valid_dataset,
    batch_size=BATCH_SIZE,
    cats=cat_names,
    conts=con_names,
    labels=['action'],
    parts_per_chunk=PARTS_PER_CHUNK
)

In [ ]:
def gen_col(batch):
    return (batch[0], batch[1], batch[2].long())

In [ ]:
train_dataloader = DLDataLoader(train_data_itrs, collate_fn=gen_col, batch_size=None, pin_memory=False, num_workers=0)
valid_dataloader = DLDataLoader(valid_data_itrs, collate_fn=gen_col, batch_size=None, pin_memory=False, num_workers=0)
databunch = TabularDataLoaders(train_dataloader, valid_dataloader)

In [ ]:
embeddings = list(get_embedding_sizes(workflow).values())
embeddings

In [ ]:
model = TabularModel(emb_szs=embeddings, n_cont=len(con_names), out_sz=2, layers=[512, 256]).cuda()
learn =  Learner(databunch, model, loss_func = torch.nn.CrossEntropyLoss(), metrics=[accuracy], cbs=ProgressCallback())

In [ ]:
from fastai.callback.schedule import fit_one_cycle

In [ ]:
train.head()

In [ ]:
from fastai.tabular.all import *

In [ ]:
learn.lr_find()

In [ ]:
learning_rate = 1.32e-2
epochs = 5
learn.fit(epochs, learning_rate)
fit_one_cycle(learn, n_epoch=epochs, lr_max=learning_rate)


In [ ]:
gc.collect()

In [ ]:
learn.lr_find()


In [ ]:
corr_mat = train[features].corr('spearman')

In [ ]:
select_f = corr_mat[abs(corr_mat)>=0.5]

In [ ]:
miss_X = train[miss_idx]
miss_mat = miss_X.corr()
miss_f = miss_mat[abs(miss_mat)>=0.5]

In [ ]:
miss_f = pd.read_csv("../input/janestreet/miss_f.csv")

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np


dic.to_csv('dic.csv')

def create_download_link(title = "Download CSV file", filename = "dic.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='dic.csv')

In [ ]:
import torch
import torch.nn as nn

# Below we define a neural net with 2 hidden layers to predict missing values

In [ ]:
gc.collect()

In [ ]:
train = 

In [ ]:
imputer = KNNImputer(weights=wt)

In [ ]:
train[features].isna().sum().sum()

In [ ]:
gc.collect()

In [ ]:
class missnet():
    def __init__(self,x,y):
        self.y = y
        self.x = x
        self.data = x.merge(y,left_index=True,right_index=True)
        
        
    def forward(self):
        knn_imputer = impute.KNNImputer()
        knn_imputer.fit_transform(self.x)
        train_y = self.y[self.y!=np.nan]
        predict_y = self.
        
        
        
    

In [ ]:
type(train.index)

In [ ]:
train.ts_id

In [ ]:
train.set_index(train.ts_id,inplace=True)


In [ ]:
data = pd.read_csv("../input/jane-street-market-prediction/train.csv")

In [ ]:
data.to_hdf(path_or_buf="./data_hdf5_format",key="data_hdf5_format")

In [ ]:

gc.collect()

In [ ]:
a=train.isna().sum().sort_values(ascending=False).to_dict()

In [ ]:
tag.shape

In [ ]:
a

In [ ]:
gc.collect()

In [ ]:
train.feature_12[:72]

In [ ]:
for y in miss_X:
    "X_{}".format(y) = miss_f[y]

In [ ]:
procs = [Categorify, FillMissing, Normalize]
splits = (L(np.arange(1000000), use_list=True),
          L(np.arange(1000000, 1981287), use_list=True))

In [ ]:
splits

In [ ]:
to = TabularPandas(train, procs, cat_col, cont_col, y_names="action", splits=splits)
dls = to.dataloaders(bs=16384)